In [27]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Tout le travail API


In [28]:
#API key
api_key = "RGAPI-46165a21-b349-4a50-9e4b-9bf71bc5d784" #Compte Slylio 
api_key2 = "RGAPI-ee90aa41-8d9f-44ed-8665-367ebfc3435c" #Compte emilio
api_key3 = "RGAPI-a67a5383-9867-4742-b38c-a606875bede1" #Compte projetML
api_key5 = "RGAPI-0b66c15d-25e1-45be-9c2b-277acaafc087" #Comte alex1
api_key6 = "RGAPI-55f78a5b-8732-4422-8820-19c07fe793cb" #Compte alex2

In [29]:
"""_summary_
    This function returns the summonerId of master + ranks players in europe west.
    
    _description_
    This function returns the summonerId of the player by using the API key.
    
    _parameters_
    api_key : str
        The API key that is used to authenticate the request
"""

def get_summonerId(api_key):
    challenger_url = 'https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Challanger ENDPOINT
    grandmaster_url = 'https://euw1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Grandmaster EDNPOINT
    master_url = 'https://euw1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Master ENDPOINT

    league_url_lists = [challenger_url, grandmaster_url, master_url] # Creating a list with the 3 endpoints (master, grandmaster and challenger)

    df_list = [] # A list that will store the response from the get request (league_df)

    for url in league_url_lists: # Loop interation through league_url_lists
        resp = requests.get(url) # Sending a get request to the url
        league_df = pd.json_normalize(resp.json()['entries'])
        df_list.append(league_df)
    
    final_league_df = pd.concat(df_list) # Concatenating the stored information in league_url_lists to create the final_league_df

    return final_league_df

In [30]:
league_df = get_summonerId(api_key)
print('league_df has information about {} players between master and challenger elo'.format(league_df.shape[0]))

league_df has information about 18511 players between master and challenger elo


In [19]:
league_df

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,M8MvHAZFAOYfeqTr5ybyKzOrY8PsVXqf4zrTHsUJG9BSmvA,Byeongshin,1208,I,132,79,False,False,False,True
1,wNSYM583sQivXGs1aqG6YXY85vUeH9lgLne3AVlKWNjIwz...,Fleshyy,1153,I,203,149,True,False,False,False
2,jKDB8bKTUgkVAzh8IrN8f4kWUcjFWaW3w9S0L9aELgjYOCE,Furuy,887,I,174,120,False,False,True,False
3,aIs_QexMiogaRL36rImTnIhg9SGwHm7UEuryMzq9ZTDjkTI,pocovirtuoso,861,I,202,159,True,False,False,False
4,sYkkZj3_jS1-yRfpIVl4oY_lWHx5sPwQj-Bbujbsyiq4LDs,McBooshi,810,I,201,158,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...
17409,5uInWMkL3QVbcPyD6F4e_gUjFqO9D47AfhSAjNY9VzC7mgo,Topete,123,I,85,71,False,False,False,True
17410,uq2Fwa08Ua9m1aG86OeB7U_i3ub_LLhG9VoU5Txj8nTrqm4,Empoggers,349,I,42,22,False,False,False,True
17411,f2sYnwYPzwivtJ12Va0x5Zq3NDSZqNigWeVsw6Dbp6896H...,Lofi Suppie,1,I,21,11,False,False,True,True
17412,37lKbdlV31prMMPJ1qnFe40Yrqw2OzAXLkpBEcCDMreyrGaJ,Big DICK zilean,1,I,134,137,False,False,True,False


In [20]:
def get_summonerinfo_by_summonerId(df, api_key):
    
    summoner_list = [] # list that will store the each summoner_info DataFrame

    for i in range(len(df)): # len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        # Creating a dynamic api_ulr for each register in the summonerId column using iloc
        api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
        print(api_url)
        # Repeating the process we did at the begining
        resp = requests.get(api_url)
        print(resp.status_code)

        # if, elif and while true to deal with riot api rate limit
        if resp.status_code == 200:
            pass
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 10 second') #approximate 10 second wait before trying again
                    time.sleep(10)

                    # Trying again
                    print(api_url)
                    api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break

        summoner_info = pd.json_normalize(resp.json()) # The same process we did at the beggining to create the DataFrame
        summoner_list.append(summoner_info) # Appending the DataFrame generated (summoner_info) into the list "summoner_list"
        
    final_summoner_info = pd.concat(summoner_list) # Using pd.concat to concatenate the results stored in the list "summoner_list"
            
    return final_summoner_info
    

In [21]:
"""
summoner_info = get_summonerinfo_by_summonerId(league_df[:1500], api_key)
summoner_info2 = get_summonerinfo_by_summonerId(league_df[1500:3000], api_key)
summoner_info3 = get_summonerinfo_by_summonerId(league_df[3000:5500], api_key)
summoner_info4 = get_summonerinfo_by_summonerId(league_df[5500:8000], api_key)
summoner_info5 = get_summonerinfo_by_summonerId(league_df[8000:10500], api_key)
summoner_info = pd.concat([summoner_info, summoner_info2, summoner_info3, summoner_info4, summoner_info5])
summoner_info.drop_duplicates(subset=['accountId'], inplace=True)
summoner_info.to_csv('summoner_info.csv', index=False)
"""

"\nsummoner_info = get_summonerinfo_by_summonerId(league_df[:1500], api_key)\nsummoner_info2 = get_summonerinfo_by_summonerId(league_df[1500:3000], api_key)\nsummoner_info3 = get_summonerinfo_by_summonerId(league_df[3000:5500], api_key)\nsummoner_info4 = get_summonerinfo_by_summonerId(league_df[5500:8000], api_key)\nsummoner_info5 = get_summonerinfo_by_summonerId(league_df[8000:10500], api_key)\nsummoner_info = pd.concat([summoner_info, summoner_info2, summoner_info3, summoner_info4, summoner_info5])\nsummoner_info.drop_duplicates(subset=['accountId'], inplace=True)\nsummoner_info.to_csv('summoner_info.csv', index=False)\n"

In [22]:
def get_matchids_by_puuid(df, api_key):

    matchids_list = []

    for i in range(len(df)):

        api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=50&api_key={}'.format(df['puuid'].iloc[i], api_key)

        resp = requests.get(api_url) # Sending a get request to the api_url
        print(resp.json())

        if resp.status_code == 200:
            matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
            matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 100 second') #approximate 110 second wait
                    time.sleep(10)

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=50&api_key={}'.format(df['puuid'].iloc[i], api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                    if resp.status_code == 200:
                        matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
                        matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
                        print('limit cost resolve')
                        break
                else:
                    break

    matchids_df = pd.concat(matchids_list) # Using pd.concat to concatenate the results stored in the "matchids_list"
    
    return matchids_df


In [23]:
matchids_df  = get_matchids_by_puuid(summoner_info, api_key)

NameError: name 'summoner_info' is not defined

In [24]:
#Remove duplicates
size_before = matchids_df.shape[0]
matchids_df = matchids_df.drop_duplicates()
size_after = matchids_df.shape[0]
print('Removed {} duplicates'.format(size_before - size_after))
print('matchids_df has information about {} matches'.format(matchids_df.shape[0]))

NameError: name 'matchids_df' is not defined

In [25]:
matchids_df.to_csv('matchids.csv', index=False)

NameError: name 'matchids_df' is not defined

In [26]:
matchids_df.reset_index(inplace=True)
matchids_df.drop(columns = 'index', inplace = True )
matchids_df

NameError: name 'matchids_df' is not defined

In [33]:
def get_match_info_by_matchId(match_ids, api_key):

    matchid_info_list = [] # List that will store the dataframes generated, for each match id in the for loop 

    for i in range(len(match_ids)):# len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        api_ulr = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}/timeline?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # here we use the iloc function as explained above to form the api link for each matchid
        resp = requests.get(api_ulr) # Sending a get request to the api_url
        print(resp.status_code) # Printing the status_code of the request

        if resp.status_code == 200: # Here we need to use if statment, because of the riot API rate limit which is 20 requests every 1 sec and 100 requests every 2 minutes. A 200 status_code means that the request was successful so it can move on.
            pass                        
        elif resp.status_code == 429: # A 429 means that we have exceeded the rate limit. To solve that we need to wait and then try again to get a successful request
            while True: # To do taht we use a while true loop, to execute the if, elif statment repeatdly until the status_code is 200. After that we use break statement to come out of this loop.
                if resp.status_code == 429: 
                    print('429 delay try 10 second') #approximate 110 second wait to work
                    time.sleep(10) # Sleep

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # Trying the request again
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break        

        # Same process we did in the beginning             
        matchid_info = pd.json_normalize(resp.json())
        matchid_info_list.append(matchid_info) # Merging all the dataframes generated and then stored in the matchid_info_list with the concat() function
        
    matchid_info_df = pd.concat(matchid_info_list)

    return matchid_info_df

In [35]:
matchs_ids = pd.read_csv('matchids.csv')

In [37]:
matchs_info_df = get_match_info_by_matchId(matchs_ids[:10], api_key)

200
200
200
200
200
200
200
200
200
200


In [38]:
matchs_info_df

,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.frameInterval,info.frames,info.gameId,info.participants
0,2,EUW1_6872766682,[YKfu3PpMYE512P6wajKIDehjOhXJ_En_S76jRiM0rnIaJ...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711467506447, ...",6872766682,"[{'participantId': 1, 'puuid': 'YKfu3PpMYE512P..."
0,2,EUW1_6872721234,[KITc75brBGxk4AD7QiIanRsb6x_1vlUgGv4-VoquZfbRN...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711465572396, ...",6872721234,"[{'participantId': 1, 'puuid': 'KITc75brBGxk4A..."
0,2,EUW1_6872682077,[WnQj4bd9VYc4rDxJldB4RU0m8O7e0NCJEECU_KAWCXyPb...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711463416185, ...",6872682077,"[{'participantId': 1, 'puuid': 'WnQj4bd9VYc4rD..."
0,2,EUW1_6872636755,[YKfu3PpMYE512P6wajKIDehjOhXJ_En_S76jRiM0rnIaJ...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711460683572, ...",6872636755,"[{'participantId': 1, 'puuid': 'YKfu3PpMYE512P..."
0,2,EUW1_6872581293,[cYEEIca3obWvxIm-_nioVJ1N6QJXX9h9EXf7xgK3hhLoB...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711457894967, ...",6872581293,"[{'participantId': 1, 'puuid': 'cYEEIca3obWvxI..."
0,2,EUW1_6872545071,[0WM-W1qeYmyLlQIMAEpcOa9ntapJRHEw6U88OiBCe7WSm...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711455364362, ...",6872545071,"[{'participantId': 1, 'puuid': '0WM-W1qeYmyLlQ..."
0,2,EUW1_6872513833,[ZTkLh8teTm41d_1LoDWqq_cT4MedIaNxABNVYKi_5ZpWB...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711453029595, ...",6872513833,"[{'participantId': 1, 'puuid': 'ZTkLh8teTm41d_..."
0,2,EUW1_6872481322,[sh5UxZ1I0YaZA3yTy0Mudxz4ob9uiaqcRoDNdnAtkM07I...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711451060015, ...",6872481322,"[{'participantId': 1, 'puuid': 'sh5UxZ1I0YaZA3..."
0,2,EUW1_6871194869,[CclAQIiyGIHheh_cNpk5iPk675FgEASgHo-tD--YyS1Ti...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711370269192, ...",6871194869,"[{'participantId': 1, 'puuid': 'CclAQIiyGIHheh..."
0,2,EUW1_6871155439,[DcZ2LrRLa9u4CEDll5ULX9kB9diRD0jO9216UAIQ19wsI...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711368164092, ...",6871155439,"[{'participantId': 1, 'puuid': 'DcZ2LrRLa9u4CE..."


In [32]:
from concurrent.futures import ThreadPoolExecutor

In [31]:
matchs_ids = pd.read_csv('matchids.csv')

In [32]:
matchs_ids

,matchid
0,EUW1_6872766682
1,EUW1_6872721234
2,EUW1_6872682077
3,EUW1_6872636755
4,EUW1_6872581293
...,...
237638,EUW1_6811998207
237639,EUW1_6811267636
237640,EUW1_6811199346
237641,EUW1_6810916271


In [37]:
def multi_get_match_info(matchs_ids, api_keys):
    with ThreadPoolExecutor() as executor:
        futures = []
        matchs_info_df = []
        for i in range(len(api_keys)):
            future = executor.submit(get_match_info_by_matchId, np.array_split(matchs_ids, len(api_keys))[i], api_keys[i])
            futures.append(future)
        for future in futures:
            matchs_info_df.append(future.result())
    return pd.concat(matchs_info_df)

    
#Create a 5Gb file
df_matchs_info1 = multi_get_match_info(matchs_ids[:10000], [api_key, api_key2, api_key3, api_key5, api_key6])
df_matchs_info1.to_csv('matchs_info1.csv', index=False)

#df_matchs_info2 = multi_get_match_info(matchs_ids[10000:], [api_key, api_key2, api_key3, api_key4, api_key5, api_key6])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [36]:
df_matchs_info1.shape
df_matchs_info1

,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.frameInterval,info.frames,info.gameId,info.participants
0,2,EUW1_6872766682,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711467506447, ...",6872766682,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi..."
0,2,EUW1_6872721234,[nFVx2BkBRefMe_OoGq9ATC9zKDtMRxC9s8PGd8oTu61aL...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711465572396, ...",6872721234,"[{'participantId': 1, 'puuid': 'nFVx2BkBRefMe_..."
0,2,EUW1_6872682077,[lreOyIqCn3xo20ipRzSwUTvwqW8eFxA0l8iXGni3OM9Rp...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711463416185, ...",6872682077,"[{'participantId': 1, 'puuid': 'lreOyIqCn3xo20..."
0,2,EUW1_6872636755,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711460683572, ...",6872636755,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi..."
0,2,EUW1_6872581293,[wCdvPlukkQSkihDiNIfb5Bkez12fl-hScsUtlhCWgJ1Ib...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711457894967, ...",6872581293,"[{'participantId': 1, 'puuid': 'wCdvPlukkQSkih..."
...,...,...,...,...,...,...,...,...
0,2,EUW1_6853619933,[Vs-geAQLyiP-nzjTaSauCBvKeTuYocpvVzcJSJNNLmgCV...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710269234617, ...",6853619933,"[{'participantId': 1, 'puuid': 'Vs-geAQLyiP-nz..."
0,2,EUW1_6853580203,[dNZSHBR_g4lA99Ijd1v7PvJqo41F_Zsvu7bULMVY8PdEQ...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710267734792, ...",6853580203,"[{'participantId': 1, 'puuid': 'dNZSHBR_g4lA99..."
0,2,EUW1_6853508233,[IyJ6wYXG_hPbnaWFhPQaGj-MeqoFbwpI9Jgu-2CcHao8C...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710264489863, ...",6853508233,"[{'participantId': 1, 'puuid': 'IyJ6wYXG_hPbna..."
0,2,EUW1_6853320591,[RriPQqhDxD9XwX0_YZbSfp48tPQoW3x2rEiXm1Vur49jV...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710253783590, ...",6853320591,"[{'participantId': 1, 'puuid': 'RriPQqhDxD9XwX..."


In [ ]:
df_matchs_info1[:1000].to_csv('matchs_info1-light.csv', index=False)

In [19]:
data = pd.read_csv("matchs_info1.csv")

In [6]:
game1 = data.loc[0]

In [22]:
game1["info.frames"]

AttributeError: 'str' object has no attribute 'to_json'

In [26]:
from pandas import to_parquet
pd.to_parquet(game1)

ImportError: cannot import name 'to_parquet' from 'pandas' (c:\Users\lfran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\__init__.py)

In [67]:
matchs_info_df_index = matchs_info_df.reset_index()
matchs_info_df_index

,index,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.frameInterval,info.frames,info.gameId,info.participants
0,0,2,EUW1_6872766682,[YKfu3PpMYE512P6wajKIDehjOhXJ_En_S76jRiM0rnIaJ...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711467506447, ...",6872766682,"[{'participantId': 1, 'puuid': 'YKfu3PpMYE512P..."
1,0,2,EUW1_6872721234,[KITc75brBGxk4AD7QiIanRsb6x_1vlUgGv4-VoquZfbRN...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711465572396, ...",6872721234,"[{'participantId': 1, 'puuid': 'KITc75brBGxk4A..."
2,0,2,EUW1_6872682077,[WnQj4bd9VYc4rDxJldB4RU0m8O7e0NCJEECU_KAWCXyPb...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711463416185, ...",6872682077,"[{'participantId': 1, 'puuid': 'WnQj4bd9VYc4rD..."
3,0,2,EUW1_6872636755,[YKfu3PpMYE512P6wajKIDehjOhXJ_En_S76jRiM0rnIaJ...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711460683572, ...",6872636755,"[{'participantId': 1, 'puuid': 'YKfu3PpMYE512P..."
4,0,2,EUW1_6872581293,[cYEEIca3obWvxIm-_nioVJ1N6QJXX9h9EXf7xgK3hhLoB...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711457894967, ...",6872581293,"[{'participantId': 1, 'puuid': 'cYEEIca3obWvxI..."
5,0,2,EUW1_6872545071,[0WM-W1qeYmyLlQIMAEpcOa9ntapJRHEw6U88OiBCe7WSm...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711455364362, ...",6872545071,"[{'participantId': 1, 'puuid': '0WM-W1qeYmyLlQ..."
6,0,2,EUW1_6872513833,[ZTkLh8teTm41d_1LoDWqq_cT4MedIaNxABNVYKi_5ZpWB...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711453029595, ...",6872513833,"[{'participantId': 1, 'puuid': 'ZTkLh8teTm41d_..."
7,0,2,EUW1_6872481322,[sh5UxZ1I0YaZA3yTy0Mudxz4ob9uiaqcRoDNdnAtkM07I...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711451060015, ...",6872481322,"[{'participantId': 1, 'puuid': 'sh5UxZ1I0YaZA3..."
8,0,2,EUW1_6871194869,[CclAQIiyGIHheh_cNpk5iPk675FgEASgHo-tD--YyS1Ti...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711370269192, ...",6871194869,"[{'participantId': 1, 'puuid': 'CclAQIiyGIHheh..."
9,0,2,EUW1_6871155439,[DcZ2LrRLa9u4CEDll5ULX9kB9diRD0jO9216UAIQ19wsI...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711368164092, ...",6871155439,"[{'participantId': 1, 'puuid': 'DcZ2LrRLa9u4CE..."


In [90]:
pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"])-1]).iloc[len(pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"])-1]))-1]["winningTeam"]

200.0

In [61]:
pd.json_normalize(matchs_info_df_index["info.frames"][0][0])

,events,timestamp,participantFrames.1.championStats.abilityHaste,participantFrames.1.championStats.abilityPower,participantFrames.1.championStats.armor,participantFrames.1.championStats.armorPen,participantFrames.1.championStats.armorPenPercent,participantFrames.1.championStats.attackDamage,participantFrames.1.championStats.attackSpeed,participantFrames.1.championStats.bonusArmorPenPercent,...,participantFrames.10.goldPerSecond,participantFrames.10.jungleMinionsKilled,participantFrames.10.level,participantFrames.10.minionsKilled,participantFrames.10.participantId,participantFrames.10.position.x,participantFrames.10.position.y,participantFrames.10.timeEnemySpentControlled,participantFrames.10.totalGold,participantFrames.10.xp
0,"[{'realTimestamp': 1711467506447, 'timestamp':...",0,0,0,36,0,0,25,100,0,...,0,0,1,0,10,14281,14199,0,500,0


In [126]:
dataset = pd.DataFrame()
for i in range(len(matchs_info_df_index)):
    end = pd.DataFrame(pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"])-1]).iloc[len(pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"])-1]))-1]).transpose()
    #to do assert
    end["winningTeam"] = int((end["winningTeam"]-100)/100)
    end = pd.DataFrame(end.reset_index()["winningTeam"])

    match_data_stats = pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][i])["participantFrames"])
    for j in range(11):
        match_data_stats.columns = match_data_stats.columns.str.removeprefix(str(j) + ".")
    team1_stats = match_data_stats.iloc[:, 1:240]
    team2_stats = match_data_stats.iloc[:, 240:]
    team1_stats = team1_stats.groupby(team1_stats.columns, axis=1).sum()
    team1_stats = team1_stats.add_prefix("Team1.")
    team2_stats = team2_stats.groupby(team2_stats.columns, axis=1).sum()
    team2_stats = team2_stats.add_prefix("Team2.")
    match_data_stats_teams = pd.concat([team1_stats, team2_stats], axis=1)
    match_data_stats_teams
    match_data_stats_teams_15 = match_data_stats_teams.loc[15]
    match_data_stats_teams_15= pd.DataFrame(match_data_stats_teams_15).transpose().reset_index().drop(["index"], axis=1)
    pd.concat([end, match_data_stats_teams_15], axis=1)
    dataset = pd.concat([dataset, pd.concat([end, match_data_stats_teams_15], axis=1)], axis = 0)
dataset

,winningTeam,Team1.championStats.abilityHaste,Team1.championStats.abilityPower,Team1.championStats.armor,Team1.championStats.armorPen,Team1.championStats.armorPenPercent,Team1.championStats.attackDamage,Team1.championStats.attackSpeed,Team1.championStats.bonusArmorPenPercent,Team1.championStats.bonusMagicPenPercent,...,Team2.goldPerSecond,Team2.jungleMinionsKilled,Team2.level,Team2.minionsKilled,Team2.participantId,Team2.position.x,Team2.position.y,Team2.timeEnemySpentControlled,Team2.totalGold,Team2.xp
0,1,0,177,352,0,0,628,817,0,0,...,50,100,47,421,40,58227,25880,1049805,24603,29860
0,1,0,185,446,0,0,551,618,0,0,...,50,95,46,402,40,44645,41057,395210,27729,30281
0,1,0,186,466,0,0,603,826,0,0,...,50,104,49,389,40,39313,27933,730043,31166,31966
0,0,0,195,307,0,0,746,945,0,0,...,50,107,48,385,40,43743,50106,957390,30368,32863
0,1,0,230,348,0,30,759,635,0,0,...,50,83,46,370,40,49966,24051,725235,28765,29713
0,0,0,177,343,0,0,674,887,0,0,...,50,88,45,362,40,63592,22483,624787,25432,26804
0,1,0,323,407,0,0,558,775,0,0,...,50,90,43,346,40,54002,45009,584964,26072,26578
0,0,0,231,396,0,0,586,1048,0,0,...,50,91,46,368,40,54895,32448,419561,28198,29570
0,0,0,269,369,0,30,516,665,0,0,...,50,102,45,379,40,41302,43276,700508,25532,27479
0,1,0,113,386,0,0,632,694,0,0,...,50,77,45,391,40,23450,54880,848248,30680,29680


In [128]:
dataset.to_csv("maches_15m_data.csv", index = False)

In [100]:
end = pd.DataFrame(pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"])-1]).iloc[len(pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][0])["events"])-1]))-1]).transpose()
#to do assert
end["winningTeam"] = int((end["winningTeam"]-100)/100)
end = pd.DataFrame(end.reset_index()["winningTeam"])
end


match_data_info_frames = pd.json_normalize(resp.json()['info']['frames'])
match_data_event= match_data_info_frames["events"]
match_data_stats = match_data_info_frames.drop(["events"], axis=1)


match_data_stats.columns = match_data_stats.columns.str.removeprefix('participantFrames.')
for i in range(11):
    match_data_stats.columns = match_data_stats.columns.str.removeprefix(str(i) + ".")

team1_stats = match_data_stats.iloc[:, 1:241]
team2_stats = match_data_stats.iloc[:, 241:]
team1_stats = team1_stats.groupby(team1_stats.columns, axis=1).sum()
team1_stats = team1_stats.add_prefix("Team1.")
team2_stats = team2_stats.groupby(team2_stats.columns, axis=1).sum()
team2_stats = team2_stats.add_prefix("Team2.")

match_data_stats_teams = pd.concat([team1_stats, team2_stats], axis=1) 
match_data_stats_teams.drop(29, axis=0)
match_data_stats_teams = match_data_stats_teams
if len(match_data_stats_teams) < 16:
    match_data_stats_teams = None
match_data_stats_teams = match_data_stats_teams.head(16)
match_data_stats_teams_15 = pd.DataFrame(match_data_stats_teams.loc[15]).transpose().reset_index().drop(["index"], axis=1)
pd.concat([match_data_stats_teams_15, end], axis=0)

#if len(match_data_stats_teams) < 16: vire


KeyError: 'events'